---
title: "dists"
---

# CMF


In [ ]:
import bokeh.io
bokeh.io.output_notebook(hide_banner=True)

import numpy as np
import scipy.stats as st
import scipy.special

import bokeh.plotting
import bokeh.application
import bokeh.application.handlers


x = np.arange(0, 11)
x_size = x[-1] - x[0]
x_c = np.empty(2*len(x))
x_c[::2] = x
x_c[1::2] = x
x_c = np.concatenate(((max(x[0] - 0.05*x_size, x[0] - 0.95),), 
                      x_c,
                      (min(x[-1] + 0.05*x_size, x[-1] + 0.95),)))
x_cdf = np.concatenate(((x_c[0],), x))

y = st.binom.cdf(x_cdf, 10, 0.5)
y_c = np.empty_like(x_c)
y_c[::2] = y
y_c[1::2] = y

p = bokeh.plotting.figure(min_height=250,
                          min_width=350,
                          x_axis_label='n',
                          y_axis_label='F(n; 10, 0.5)')
p.line(x_c, y_c, line_width=2)
bokeh.io.show(p)

# CMF


In [ ]:
bokeh.io.output_notebook(hide_banner=True)
x = np.arange(0, 11)
y = st.binom.cdf(x, 10, 0.5)

p = bokeh.plotting.figure(min_height=250,
                          min_width=350,
                          x_axis_label='n',
                          y_axis_label='F(n; 10, 0.5)')
p.segment(x[:-1], y[:-1], x[1:], y[:-1], line_width=2)
p.ray(0, 0, angle=np.pi, length=0, line_width=2)
p.ray(x[-1], 1, angle=0, length=0, line_width=2)
p.circle([0], [0], fill_color='white')
p.circle(x[1:], y[:-1], fill_color='white')
p.circle(x, y)
bokeh.io.show(p)

# Linked plots


In [ ]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.sampledata.penguins import data
from bokeh.transform import factor_cmap
bokeh.io.output_notebook(hide_banner=True)

SPECIES = sorted(data.species.unique())

TOOLS = "box_select,lasso_select,help"

source = ColumnDataSource(data)

left = figure(width=300, height=400, title=None, tools=TOOLS,
              background_fill_color="#fafafa")
left.scatter("bill_length_mm", "body_mass_g", source=source,
             color=factor_cmap('species', 'Category10_3', SPECIES))

right = figure(width=300, height=400, title=None, tools=TOOLS,
               background_fill_color="#fafafa", y_axis_location="right")
right.scatter("bill_depth_mm", "body_mass_g", source=source,
              color=factor_cmap('species', 'Category10_3', SPECIES))

show(gridplot([[left, right]]))

In [ ]:
import numpy as np
import scipy.stats as st

from bokeh.plotting import figure, show
bokeh.io.output_notebook(hide_banner=True)

rng = np.random.default_rng()
x = rng.normal(loc=0, scale=1, size=1000)

p = figure(width=670, height=400, toolbar_location=None,
           title="Normal (Gaussian) Distribution")

# Histogram
bins = np.linspace(-3, 3, 40)
hist, edges = np.histogram(x, density=True, bins=bins)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
         fill_color="skyblue", line_color="white",
         legend_label="1000 random samples")

# Probability density function
x = np.linspace(-3.0, 3.0, 100)
pdf = np.exp(-0.5*x**2) / np.sqrt(2.0*np.pi)
p.line(x, pdf, line_width=2, line_color="navy",
       legend_label="Probability Density Function")

p.y_range.start = 0
p.xaxis.axis_label = "x"
p.yaxis.axis_label = "PDF(x)"

show(p)

```{javascript}

function betacf(x, a, b) {
    // From Andreas Madsen's mathfn, Copyright (c) 2013 Andreas Madsen
    // Computes incomplete beta function as a continues fraction
    var fpmin = 1e-30,
        m = 1,
        m2, aa, c, d, del, h, qab, qam, qap;
    // These q's will be used in factors that occur in the coefficients
    qab = a + b;
    qap = a + 1;
    qam = a - 1;
    c = 1;
    d = 1 - qab * x / qap;
    if (Math.abs(d) < fpmin) d = fpmin;
    d = 1 / d;
    h = d;
    for (; m <= 100; m++) {
        m2 = 2 * m;
        aa = m * (b - m) * x / ((qam + m2) * (a + m2));
        // One step (the even one) of the recurrence
        d = 1 + aa * d;
        if (Math.abs(d) < fpmin) d = fpmin;
        c = 1 + aa / c;
        if (Math.abs(c) < fpmin) c = fpmin;
        d = 1 / d;
        h *= d * c;
        aa = -(a + m) * (qab + m) * x / ((a + m2) * (qap + m2));
        // Next step of the recurrence (the odd one)
        d = 1 + aa * d;
        if (Math.abs(d) < fpmin) d = fpmin;
        c = 1 + aa / c;
        if (Math.abs(c) < fpmin) c = fpmin;
        d = 1 / d;
        del = d * c;
        h *= del;
        if (Math.abs(del - 1.0) < 3e-7) break;
    }
    return h;
}


function regularized_incomplete_beta(x, a, b) {
    // From Andreas Madsen's mathfn, Copyright (c) 2013 Andreas Madsen
    // Computes incomplete beta function as a continued fraction
    if (x < 0 || x > 1) {
        throw new RangeError('First argument must be between 0 and 1.');
    }

    // Special cases, there can make trouble otherwise
    else if (a === 1 && b === 1) return x;
    else if (x === 0) return 0;
    else if (x === 1) return 1;
    else if (a === 0) return 1;
    else if (b === 0) return 0;

    else {
        var bt = Math.exp(lngamma(a + b) - lngamma(a) - lngamma(b) + a * Math.log(x) + b * log1p(-x));

        // Use continued fraction directly.
        if (x < (a + 1) / (a + b + 2)) return bt * betacf(x, a, b) / a;

        // else use continued fraction after making the symmetry transformation.
        else return 1 - bt * betacf(1 - x, b, a) / b;
    }
}


function incomplete_beta(x, a, b) {
    return regularized_incomplete_beta(x, a, b) * Math.exp(lnbeta(a, b));
}
```

In [ ]:
import numpy as np
import scipy.special
import scipy.stats as st


from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
bokeh.io.output_notebook(hide_banner=True)


def make_plot(title, hist, edges, x, pdf, cdf):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.line(x, pdf, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
    p.line(x, cdf, line_color="orange", line_width=2, alpha=0.7, legend_label="CDF")

    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

# Normal Distribution

mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1 = make_plot("Normal Distribution (μ=0, σ=0.5)", hist, edges, x, pdf, cdf)
show(gridplot([p1], ncols=1, width=400, height=400, toolbar_location=None))

# LogNormal

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

# Log-Normal Distribution

mu, sigma = 0, 0.5

measured = np.random.lognormal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0.0001, 8.0, 1000)
pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((np.log(x)-mu)/(np.sqrt(2)*sigma)))/2

p2 = make_plot("Log Normal Distribution (μ=0, σ=0.5)", hist, edges, x, pdf, cdf)
show(gridplot([p2], ncols=1, width=400, height=400, toolbar_location=None))

# Gamma

The exponential distribution predicts the wait time until the *very first* event. The gamma distribution, on the other hand, predicts the wait time until the *k-th* event occurs.

The amount of time we have to wait for α  arrivals of a Poisson process. More concretely, if we have events, $X_1 , X_2 , …, X_α$ that are exponentially distributed, $X_1+X_2+⋯+X_α$  is Gamma distributed.

## Example
Any multi-step process where each step happens at the same rate. This is common in molecular rearrangements.

## [Gamma1](http://www.probonto.org/ontology#PROB_k0000572) shape, scale prameterization

- shape ($k < 0$), 
- scale ($\theta > 0$)
- Variance $$\mathbb{V}ar[Gamma_1|k,\theta] = k \theta^2$${#eq-gamma2-pdf}
- PDF $$\frac{1}{\Gamma(k) \theta^k} x^{k \,-\, 1}e^{-\frac{x}{\theta}}$${#eq-gamma1-pdf}
- CDF $$\frac{1}{\Gamma(k)} \gamma\left(k,\, \frac{x}{\theta}\right)$${#eq-gamma1-cdf}
- SF $$1 - \frac{\Gamma(\beta,x/\alpha)}{\Gamma(\beta)}$${#eq-gamma1-sf}
- mode $$(k \,-\, 1)\theta \text{ for } k \;{\geq}\; 1$${#eq-gamma1-mode}
- median $$\text{No simple closed form}$${#eq-gamma1-median}
- mean $$k \theta$${#eq-gamma1-mean}


For (k, θ) parameterization: θ is a reciprocal of the event rate λ, which is the mean wait time (the average time between event arrivals).


## [Gamma2](http://www.probonto.org/ontology#PROB_k0000598) shape rate prameterization

- shape ($\alpha > 0$), 
- rate ($\beta > 0$)
- PDF $$\frac{\mu^r x^{r-1} e^{-\mu x}}{\Gamma(r)}$${#eq-gamma2-pdf}
- CDF
- Var

For (α, β) parameterization: Using our notation k (the # of events) & λ (the rate of events), simply substitute α with k, β with λ. The PDF stays the same format as what we’ve derived.


In [ ]:
bokeh.io.output_notebook(hide_banner=True)

# Gamma Distribution

k, theta = 7.5, 1.0

measured = np.random.gamma(k, theta, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0.0001, 20.0, 1000)
pdf = x**(k-1) * np.exp(-x/theta) / (theta**k * scipy.special.gamma(k))
cdf = scipy.special.gammainc(k, x/theta)

p3 = make_plot("Gamma Distribution (k=7.5, θ=1)", hist, edges, x, pdf, cdf)
show(gridplot([p3], ncols=1, width=400, height=400, toolbar_location=None))

In [ ]:
import numpy as np
from scipy.stats import gamma
import matplotlib.pyplot as plt

def plot_gamma_k():
    """
    k : the number of events for which you are waiting to occur.
    λ : the rate of events happening following Poisson dist.
    """
    x = np.linspace(0, 50, 1000)
    a_values=[1.,2.,3.,5., 9.,7.5,0.5]
    b_values=[2.,2.,2.,1.,0.5,1. ,1.]
    plt.title("PDF of Gamma Distribution")
    plt.xlabel("T")
    plt.ylabel("Probability Density")
    colors = plt.cm.Pastel1(np.linspace(0, 1, len(a_values)))
    colors = plt.cm.rainbow(np.linspace(0, 1, len(a_values)))

    for i,z in enumerate(zip(a_values,b_values)): 
      a,b = z
      #mean, var, skew, kurt = gamma.stats(a, moments='mvsk')
      y = gamma.pdf(x, a=a,scale=b)
      plt.plot(x, y, label=f"k={a}, θ={b}", color=colors[i])
      plt.legend(bbox_to_anchor=(1, 1), loc='upper right',
               borderaxespad=1, fontsize=12)
    plt.ylim([0, 0.50])
    plt.xlim([0, 20])
    plt.show()
    plt.savefig('gamma_k.png')
    plt.clf()

plot_gamma_k()

In [ ]:
def plot_gamma_lambda():
    """
    k : the number of events for which you are waiting to occur.
    λ : the rate of events happening following Poisson dist.
    """
    a = 10  # k = 10
    x = np.linspace(0, 50, 1000)
    import matplotlib as mpl

    #colors = ['gold','burlywood','darkorange']
    plt.title("PDF of Gamma Distribution (k = 10)")
    plt.xlabel("T")
    plt.ylabel("Probability Density")
    lambdas = [1,2,3]
    #colors = plt.cm.rainbow(np.linspace(0, 1, len(lambdas)))
    colors = plt.cm.Pastel1(np.linspace(0, 1, len(lambdas)))
    for i,lambda_  in enumerate(lambdas):
      #mean, var, skew, kurt = gamma.stats(a, scale=1/lambda_, moments='mvsk')
      y = gamma.pdf(x, a, scale=1/lambda_)
      plt.plot(x, y, label=f"λ = {lambda_}", color=colors[i])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=1, fontsize=12)
    plt.ylim([0, 0.40])
    plt.xlim([0, 20])
    plt.show()
    plt.savefig('gamma_lambda.png')
    plt.clf()


_=plot_gamma_lambda()

# Beta Distribution


Say you wait for two multi-step processes to happen. The individual steps of each process happen at the same rate, but the first multi-step process requires $α$ steps and the second requires $β$  steps. The fraction of the total waiting time take by the first process is Beta distributed.

## Examples:

1. The Beta Distribution can be used for representing the different probabilities as follows:

2. The likelihood of the audience rating the new movie release.
3. The click-through rate of the website, which is the proportion of visitors.
4. The conversion rate for buyers actually purchasing from your website.
5. The  survival chance of a person having blood cancer.

In [ ]:
#from numpy.random import default_rng
bokeh.io.output_notebook(hide_banner=True)
# Beta Distribution

story= """Say you wait for two multistep Poisson processes to arive. The individual steps of each process happen at the same rate, but the first multistep process requires alpha steps and the second requires beta 
 steps. The fraction of the total waiting time taken by the first process is Beta distributed."""

alpha, beta = 2, 3
# Do this (new version)
measured = np.random.default_rng().beta(alpha, beta, size=10000)
hist, edges = np.histogram(measured, density=True, bins=50)
param_vals={'alpha':1,'beta':1}
x = np.linspace(0.0001, 0.999, 1000)
#pdf = x**(k-1) * np.exp(-x/theta) / (theta**k * scipy.special.gamma(k))
rv = st.beta(alpha, beta)
pdf = rv.pdf(x)##st.beta.pdf(x,*param_vals)
cdf = rv.cdf(x)##st.beta.cdf(x,*param_vals)

p4 = make_plot("Beta Distribution (alpha=1, beta=1)", hist, edges, x, pdf, cdf)
show(gridplot([p4], ncols=1, width=400, height=400, toolbar_location=None))

# Weibull Distribution

Distribution of $x = y^\beta$ if $y\sim exp()$. For $\beta > 1$, the longer we have waited, the more likely the event is to come, and vice versa for $\beta$
.
Example. This is a model for aging. The longer an organism lives, the more likely it is to die.

In [ ]:
bokeh.io.output_notebook(hide_banner=True)
# Weibull Distribution

lam, k = 1, 1.25
measured = lam*(-np.log(np.random.uniform(0, 1, 1000)))**(1/k)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0.1, 8, 1000)
pdf = (k/lam)*(x/lam)**(k-1) * np.exp(-(x/lam)**k)
cdf = 1 - np.exp(-(x/lam)**k)
p5 = make_plot("Weibull Distribution (λ=1, k=1.25)", hist, edges, x, pdf, cdf)

show(gridplot([p5], ncols=2, width=400, height=400, toolbar_location=None))

# Pareto Distribution

Say you wait for two multistep Poisson processes to arive. The individual steps of each process happen at the same rate, but the first multistep process requires alpha steps and the second requires beta  steps. The fraction of the total waiting time taken by the first process is Beta distributed.


## Example 

- The Gutenberg-Richter Law says that the magnitudes of earthquakes in a given region are Pareto distributed. 
- size of human settlement (many small towns, a few huge cities), 
- income distribution (many poor, few obscenely rich).


In [ ]:
bokeh.io.output_notebook(hide_banner=True)
from scipy.stats import pareto

# Pareto Distribution

story= """Say you wait for two multistep Poisson processes to arive. The individual steps of each process happen at the same rate, but the first multistep process requires alpha steps and the second requires beta  steps. The fraction of the total waiting time taken by the first process is Beta distributed."""

b = 2.62
rv = st.pareto(b)
# Do this (new version)
#measured = np.random.default_rng().pareto(b, size=1000)
measured = pareto.rvs(b, size=1000)

hist, edges = np.histogram(measured, density=True, bins=50)
param_vals={'alpha':1,'beta':1}
x = np.linspace(pareto.ppf(0.01, b), pareto.ppf(0.99, b), 100)
pdf = rv.pdf(x)
cdf = rv.cdf(x)

p6 = make_plot(f"Pareto Distribution (b={b})", hist, edges, x, pdf, cdf)
show(gridplot([p6], ncols=1, width=400, height=400, toolbar_location=None))

# Generlized Pareto

In statistics, the Generalized Pareto Distribution (GPD) is a family of continuous probability distributions. It is often used to model the tails of another distribution. It is specified by three parameters: location $\mu$ , scale $\sigma$ , and shape $\x_i$


In [ ]:
bokeh.io.output_notebook(hide_banner=True)
from scipy.stats import genpareto

# Gen Pareto Distribution
c = 0.1
rv = st.genpareto(c)
# Do this (new version)
measured = genpareto.rvs(c, size=1000)

hist, edges = np.histogram(measured, density=True, bins=50)
x = np.linspace(genpareto.ppf(0.01, c), genpareto.ppf(0.99, c), 1000)
pdf = rv.pdf(x)
cdf = rv.cdf(x)

p6 = make_plot(f"Gen. Pareto Distribution (b={b})", hist, edges, x, pdf, cdf)
show(gridplot([p6], ncols=1, width=400, height=400, toolbar_location=None))

# Student t

In statistics, the Generalized Pareto Distribution (GPD) is a family of continuous probability distributions. It is often used to model the tails of another distribution. It is specified by three parameters: location $\mu$ , scale $\sigma$ , and shape $\x_i$

$$\begin{aligned}
f(y \mid \nu, \mu, \sigma) = \frac{\Gamma\left(\frac{\nu+1}{2}\right)}{\Gamma\left(\frac{\nu}{2}\right)\sqrt{\pi \nu \sigma^2}}\left(1 + \frac{(y-\mu)^2}{\nu \sigma^2}\right)^{-\frac{\nu + 1}{2}}.
\end{aligned}$${#eq-t-pdf}


In [ ]:
bokeh.io.output_notebook(hide_banner=True)
from scipy.stats import t

# Gen Pareto Distribution
nu=2 #df
mu=0.0 #loc
sigma=1.0 # scale
rv = st.t(df=nu,loc=mu,scale=sigma)
# Do this (new version)
measured = t.rvs(df=nu,loc=mu,scale=sigma, size=1000)

hist, edges = np.histogram(measured, density=True, bins=50)
x = np.linspace(t.ppf(0.01, df=nu,loc=mu,scale=sigma), t.ppf(0.99, df=nu,loc=mu,scale=sigma), 1000)
pdf = rv.pdf(x)
cdf = rv.cdf(x)

p7 = make_plot(f"Student-t (nu={nu},mu={mu},sigma={sigma})", hist, edges, x, pdf, cdf)
show(gridplot([p7], ncols=1, width=400, height=400, toolbar_location=None))

# Normal Distribution


In [ ]:
#| fig-column: screen-inset-right
#| fig-align: center 
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, Slider, TextInput,NumericInput
from bokeh.layouts import column, row
bokeh.io.output_notebook(hide_banner=True)

# Normal Distribution

mu, sigma = 0, 0.5

def make_plot(title, source, kind=[]):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    if 'pdf' in kind : 
      p.line('x', 'pdf', source=source, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
    if 'cdf' in kind :
      p.line('x', 'cdf', source=source, line_color="orange", line_width=4, alpha=0.7, legend_label="CDF")

    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

x = np.linspace(-2, 2, 1000)
measured = np.random.normal(mu, sigma, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

source = ColumnDataSource(data=dict(x=x, pdf=pdf, cdf=cdf, measured=measured))
title = f'Normal Distribution (μ={mu}, σ={sigma})'
p_p = make_plot(title=title, source=source, kind=['pdf','hist'] )
p_c = make_plot(title=title, source=source, kind=['cdf'])

## mu slider
mu_slider = Slider(start=-0.5, end=0.5, value=0.0, step=.1, title="μ")
sigma_slider = Slider(start=-0.0, end=1.5, value=0.5, step=.1, title="σ")

callback = CustomJS(args=dict(source=source,mu_slider=mu_slider,sigma_slider=sigma_slider), code="""
function gaussianRandom(mean, stdev) {
    let u = 1 - Math.random(); // Converting [0,1) to (0,1]
    let v = Math.random();
    let z = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2.0 * Math.PI * v );
    // Transform to the desired mean and standard deviation:
    return z * stdev + mean;
}

function f_pdf(x,mu,sigma){
 return 1/(sigma * Math.sqrt(2*Math.PI)) * Math.exp(-1*(x-mu)**2 / (2*sigma**2))
}

function erf(x) {
    // Error function using polynomial approximation (accurate to about 10^-7)
    var a = [1.00002368,
             0.37409196,
             0.09678418,
             -0.18628806,
             0.27886807,
             -1.13520398,
             1.48851587,
             -0.82215223,
             0.17087277];

    var t = 1 / (1 + Math.abs(x)/2);
    var expSum = -Math.pow(x, 2) - 1.26551223;

    for (var i = 0; i < a.length; i++) {
        expSum += a[i] * Math.pow(t, i+1);
    }

    var result = 1 - t * Math.exp(expSum);

    if (x < 0) return -result;
    return result;
}

function f_cdf(x,mu,sigma){
 return (1+erf((x-mu)/Math.sqrt(2*sigma**2)))/2
}

    const mu = mu_slider.value
    const sigma = sigma_slider.value
    const sdev= Math.sqrt(sigma)
    const x = source.data.x
    const pdf = Array.from(x, (x) => f_pdf(x, mu,sigma))
    const cdf = Array.from(x, (x) => f_cdf(x, mu,sigma))
    const measured = Array.from(x, (x) => gaussianRandom(mu,sdev))
    source.data = { x, pdf,cdf,measured }
""")
mu_slider.js_on_change('value', callback)
sigma_slider.js_on_change('value', callback)

# todo: update the data when mu changes
mu_min_input = NumericInput(value=-1.0,  title="min μ",width=60)
mu_min_input.js_link(attr='value',other=mu_slider, other_attr='start')
mu_max_input = NumericInput(value=1.0,  title="max μ",width=60)
mu_max_input.js_link(attr='value',other=mu_slider, other_attr='end')

sigma_min_input = NumericInput(value=0.0,  title="min σ",width=60)
sigma_min_input.js_link(attr='value',other=sigma_slider, other_attr='start')
sigma_max_input = NumericInput(value=5.0,  title="max σ",width=60)
sigma_max_input.js_link(attr='value',other=sigma_slider, other_attr='end')

p=gridplot([p_p,bokeh.layouts.Spacer(width=3),p_c], ncols=3, height=400, toolbar_location=None)

show(column(
  row(mu_min_input,mu_slider,mu_max_input),
  row(sigma_min_input,sigma_slider,sigma_max_input),
  row(p)
  ))

# PDF & CDF Interactive titles 


In [ ]:
#| fig-column: screen-inset-right
#| fig-align: center 
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, Slider, TextInput,NumericInput
from bokeh.layouts import column, row
bokeh.io.output_notebook(hide_banner=True)

# Normal Distribution

mu, sigma = 0, 0.5

def make_plot(title, source, kind=[]):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    if 'pdf' in kind : 
      p.line('x', 'pdf', source=source, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="PDF")
    if 'cdf' in kind :
      p.line('x', 'cdf', source=source, line_color="orange", line_width=4, alpha=0.7, legend_label="CDF")

    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

x = np.linspace(-2, 2, 1000)
measured = np.random.normal(mu, sigma, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

source = ColumnDataSource(data=dict(x=x, pdf=pdf, cdf=cdf, measured=measured))
title = f'Normal Distribution (μ={mu}, σ={sigma})'
p_p = make_plot(title=title, source=source, kind=['pdf','hist'] )
p_c = make_plot(title=title, source=source, kind=['cdf'])

## mu slider
mu_slider = Slider(start=-0.5, end=0.5, value=0.0, step=.1, title="μ")
sigma_slider = Slider(start=-0.0, end=1.5, value=0.5, step=.1, title="σ")

callback = CustomJS(args=dict(source=source, mu_slider=mu_slider, sigma_slider=sigma_slider, p_p=p_p, p_c=p_c), code="""
function gaussianRandom(mean, stdev) {
    let u = 1 - Math.random(); // Converting [0,1) to (0,1]
    let v = Math.random();
    let z = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2.0 * Math.PI * v );
    // Transform to the desired mean and standard deviation:
    return z * stdev + mean;
}

function f_pdf(x,mu,sigma){
 return 1/(sigma * Math.sqrt(2*Math.PI)) * Math.exp(-1*(x-mu)**2 / (2*sigma**2))
}

function erf(x) {
    // Error function using polynomial approximation (accurate to about 10^-7)
    var a = [1.00002368,
             0.37409196,
             0.09678418,
             -0.18628806,
             0.27886807,
             -1.13520398,
             1.48851587,
             -0.82215223,
             0.17087277];

    var t = 1 / (1 + Math.abs(x)/2);
    var expSum = -Math.pow(x, 2) - 1.26551223;

    for (var i = 0; i < a.length; i++) {
        expSum += a[i] * Math.pow(t, i+1);
    }

    var result = 1 - t * Math.exp(expSum);

    if (x < 0) return -result;
    return result;
}

function f_cdf(x,mu,sigma){
 return (1+erf((x-mu)/Math.sqrt(2*sigma**2)))/2
}

    const mu = mu_slider.value
    const sigma = sigma_slider.value
    const sdev= Math.sqrt(sigma)
    const x = source.data.x
    const pdf = Array.from(x, (x) => f_pdf(x, mu,sigma))
    const cdf = Array.from(x, (x) => f_cdf(x, mu,sigma))
    const measured = Array.from(x, (x) => gaussianRandom(mu,sdev))
    source.data = { x, pdf,cdf,measured }
    const title = `Normal Distribution (μ=${mu}, σ=${sigma})`
    p_p.title.text = title
    p_c.title.text = title

""")
mu_slider.js_on_change('value', callback)
sigma_slider.js_on_change('value', callback)

# todo: update the data when mu changes
mu_min_input = NumericInput(value=-1.0,  title="min μ",width=60)
mu_min_input.js_link(attr='value',other=mu_slider, other_attr='start')
mu_max_input = NumericInput(value=1.0,  title="max μ",width=60)
mu_max_input.js_link(attr='value',other=mu_slider, other_attr='end')

sigma_min_input = NumericInput(value=0.0,  title="min σ",width=60)
sigma_min_input.js_link(attr='value',other=sigma_slider, other_attr='start')
sigma_max_input = NumericInput(value=5.0,  title="max σ",width=60)
sigma_max_input.js_link(attr='value',other=sigma_slider, other_attr='end')

p=gridplot([p_p,bokeh.layouts.Spacer(width=3),p_c], ncols=3, height=400, toolbar_location=None)

show(column(
  row(mu_min_input,mu_slider,mu_max_input),
  row(sigma_min_input,sigma_slider,sigma_max_input),
  row(p)
  ))

# Picture


In [ ]:
from bokeh.plotting import figure, show, output_file
bokeh.io.output_notebook(hide_banner=True)

p = figure(x_range=(0,1), y_range=(0,1))
p.image_url(url=['bernoulli.jpg'], x=0, y=1, w=0.8, h=0.6)
show(p)